In [25]:
using JuMP
using GLPK
using CPUTime
using Statistics


In [26]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [27]:
# number of nurses
N = 15
# number of days
T = 7
# number of shifts = 3

# standard number of hours by contract per nurse
k=floor(T/7)
H_base = 36
H = [H_base*k for item in 1:N]
# maximum number of hours for each nurse
H_max = 60*k

# number of nurses required per shift and duration of each shift

R = [6; 5; 4]
h = [7; 8; 9]

# list of nurses that on the last day of the previous period covered  the night shift
p_list= [1] #can be adjusted differently
# array with the values of p per each nurse and update the dictionary with p_list
p = zeros(N)
for item in p_list
    p[item]=1
end

# wages
w = ones(N) #can be adjusted differently
# rho must be greater than max(w[i]*h[s]) for all i and all s
rho = maximum(w*h')+1

10.0

In [28]:
# create the variables
@variable(model, x[1:N, 1:3, 1:T], Bin)
@variable(model, alpha[1:3, 1:T])


3×7 Matrix{VariableRef}:
 alpha[1,1]  alpha[1,2]  alpha[1,3]  …  alpha[1,5]  alpha[1,6]  alpha[1,7]
 alpha[2,1]  alpha[2,2]  alpha[2,3]     alpha[2,5]  alpha[2,6]  alpha[2,7]
 alpha[3,1]  alpha[3,2]  alpha[3,3]     alpha[3,5]  alpha[3,6]  alpha[3,7]

In [29]:
###is_binary.(alpha)
###delete.(model, c_1)
###unregister.(model, C_1)

In [30]:
# objective function definition
@objective(model, Min, sum(x[i,s,t]*h[s]*w[i] for i in 1:N for s in 1:3 for t in 1:T) 
    + rho*sum(alpha[s,t] for s in 1:3 for t in 1:T))


7 x[1,1,1] + 7 x[1,1,2] + 7 x[1,1,3] + 7 x[1,1,4] + 7 x[1,1,5] + 7 x[1,1,6] + 7 x[1,1,7] + 8 x[1,2,1] + 8 x[1,2,2] + 8 x[1,2,3] + 8 x[1,2,4] + 8 x[1,2,5] + 8 x[1,2,6] + 8 x[1,2,7] + 9 x[1,3,1] + 9 x[1,3,2] + 9 x[1,3,3] + 9 x[1,3,4] + 9 x[1,3,5] + 9 x[1,3,6] + 9 x[1,3,7] + 7 x[2,1,1] + 7 x[2,1,2] + 7 x[2,1,3] + 7 x[2,1,4] + 7 x[2,1,5] + 7 x[2,1,6] + 7 x[2,1,7] + 8 x[2,2,1] + 8 x[2,2,2] + 8 x[2,2,3] + 8 x[2,2,4] + 8 x[2,2,5] + 8 x[2,2,6] + 8 x[2,2,7] + 9 x[2,3,1] + 9 x[2,3,2] + 9 x[2,3,3] + 9 x[2,3,4] + 9 x[2,3,5] + 9 x[2,3,6] + 9 x[2,3,7] + 7 x[3,1,1] + 7 x[3,1,2] + 7 x[3,1,3] + 7 x[3,1,4] + 7 x[3,1,5] + 7 x[3,1,6] + 7 x[3,1,7] + 8 x[3,2,1] + 8 x[3,2,2] + 8 x[3,2,3] + 8 x[3,2,4] + 8 x[3,2,5] + 8 x[3,2,6] + 8 x[3,2,7] + 9 x[3,3,1] + 9 x[3,3,2] + 9 x[3,3,3] + 9 x[3,3,4] + 9 x[3,3,5] + 9 x[3,3,6] + 9 x[3,3,7] + 7 x[4,1,1] + 7 x[4,1,2] + 7 x[4,1,3] + 7 x[4,1,4] + 7 x[4,1,5] + 7 x[4,1,6] + 7 x[4,1,7] + 8 x[4,2,1] + 8 x[4,2,2] + 8 x[4,2,3] + 8 x[4,2,4] + 8 x[4,2,5] + 8 x[4,2,6] + 8 x[4,2,7] +

In [31]:
#constraints
@constraint(model, a_1, sum(x[:,s,:] for s in 1:3).<=1 )

15×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[1,1,1] + x[1,2,1] + x[1,3,1] <= 1.0     …  x[1,1,7] + x[1,2,7] + x[1,3,7] <= 1.0
 x[2,1,1] + x[2,2,1] + x[2,3,1] <= 1.0        x[2,1,7] + x[2,2,7] + x[2,3,7] <= 1.0
 x[3,1,1] + x[3,2,1] + x[3,3,1] <= 1.0        x[3,1,7] + x[3,2,7] + x[3,3,7] <= 1.0
 x[4,1,1] + x[4,2,1] + x[4,3,1] <= 1.0        x[4,1,7] + x[4,2,7] + x[4,3,7] <= 1.0
 x[5,1,1] + x[5,2,1] + x[5,3,1] <= 1.0        x[5,1,7] + x[5,2,7] + x[5,3,7] <= 1.0
 x[6,1,1] + x[6,2,1] + x[6,3,1] <= 1.0     …  x[6,1,7] + x[6,2,7] + x[6,3,7] <= 1.0
 x[7,1,1] + x[7,2,1] + x[7,3,1] <= 1.0        x[7,1,7] + x[7,2,7] + x[7,3,7] <= 1.0
 x[8,1,1] + x[8,2,1] + x[8,3,1] <= 1.0        x[8,1,7] + x[8,2,7] + x[8,3,7] <= 1.0
 x[9,1,1] + x[9,2,1] + x[9,3,1] <= 1.0        x[9,1,7] + x[9,2,7] + x[9,3,7] <= 1.0
 x[10,1,1] + x[10,2,1] + x[10,3,1] <= 1.0     x[10,1,7] + x[10,2,7] + x[10,3,7

In [32]:
@constraint(model, b_1, sum(x[i,:,:] for i in 1:N) + alpha[:,:] .== R*(ones(T))')

3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] + x[6,1,1] + x[7,1,1] + x[8,1,1] + x[9,1,1] + x[10,1,1] + x[11,1,1] + x[12,1,1] + x[13,1,1] + x[14,1,1] + x[15,1,1] + alpha[1,1] == 6.0  …  x[1,1,7] + x[2,1,7] + x[3,1,7] + x[4,1,7] + x[5,1,7] + x[6,1,7] + x[7,1,7] + x[8,1,7] + x[9,1,7] + x[10,1,7] + x[11,1,7] + x[12,1,7] + x[13,1,7] + x[14,1,7] + x[15,1,7] + alpha[1,7] == 6.0
 x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] + x[6,2,1] + x[7,2,1] + x[8,2,1] + x[9,2,1] + x[10,2,1] + x[11,2,1] + x[12,2,1] + x[13,2,1] + x[14,2,1] + x[15,2,1] + alpha[2,1] == 5.0     x[1,2,7] + x[2,2,7] + x[3,2,7] + x[4,2,7] + x[5,2,7] + x[6,2,7] + x[7,2,7] + x[8,2,7] + x[9,2,7] + x[10,2,7] + x[11,2,7] + x[12,2,7] + x[13,2,7] + x[14,2,7] + x[15,2,7] + alpha[2,7] == 5.0
 x[1,3,1] + x[2,3,1] + x[3,3,1] + x[4,3,1] + x[5,3,1] + x[6,3,1] + x[7

In [33]:
@constraint(model, c_1, sum(x[:,s,t]*h[s] for s in 1:3 for t in 1:T).>= H)
@constraint(model, d_1, sum(x[:,s,t]*h[s] for s in 1:3 for t in 1:T).<= H_max)
@constraint(model, e_1, x[:, 3, 1:T-1] + sum(x[:,s,2:T] for s in 1:3).<= 1)
@constraint(model, f_1, sum(x[:,s,1] for s in 1:3).<= (1 .- p))
@constraint(model, g_1, alpha[:,:] .>= 0)


3×7 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 alpha[1,1] >= 0.0  alpha[1,2] >= 0.0  …  alpha[1,7] >= 0.0
 alpha[2,1] >= 0.0  alpha[2,2] >= 0.0     alpha[2,7] >= 0.0
 alpha[3,1] >= 0.0  alpha[3,2] >= 0.0     alpha[3,7] >= 0.0

In [34]:
optimize!(model)

In [35]:
println()
println("The solver stopped because: ")
@show termination_status(model)# run it in ipynb file to get userfriendly result


The solver stopped because: 
termination_status(model) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [36]:
println()
println("to see if the solver found a primal feasible point:")
@show primal_status(model)# run it in ipynb file to get userfriendly result


to see if the solver found a primal feasible point:
primal_status(model) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [37]:
println()
println("minimum cost:")
@show objective_value(model)# run it in ipynb file to get userfriendly result


minimum cost:
objective_value(model) = 839.0


839.0

In [38]:
println()
println("Matrix alpha is:")
@show value.(alpha)#run it in ipynb file to get userfriendly result


Matrix alpha is:
value.(alpha) = [0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0000000000000002 2.9999999999999973 1.0000000000000007 4.0 2.220446049250313e-16 4.0 0.0]


3×7 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0          0.0  0.0
 0.0  0.0  0.0  0.0  0.0          0.0  0.0
 1.0  3.0  1.0  4.0  2.22045e-16  4.0  0.0

In [39]:
println()
println("max(alpha[s,t] for all s and all t) is: ", maximum(value.(alpha)))



max(alpha[s,t] for all s and all t) is: 4.0


In [40]:
println()
println("sum(alpha[s,t] for all s and all t) is: ", round(value.(sum(alpha))))


sum(alpha[s,t] for all s and all t) is: 13.0


In [41]:
fault_num=0
for s in 1:3
    for t in 1:T
        if value.(alpha[s,t]) >= 0.000001
            global fault_num += 1
        end
    end
end
println("number of cases in which the number of nurses is below the required quantity is: ", fault_num)

number of cases in which the number of nurses is below the required quantity is: 5


In [42]:
println()
println("CPU Time:")
CPU_Time = @time  optimize!(model)



CPU Time:
  0.024067 seconds (343 allocations: 37.906 KiB)


In [43]:
#number of total shifts for each nurse:
println("-------------------------------------------")
for i in 1:N
    println("number of total shifts for nurse_", i, " is: ", value.(sum(x[i,s,t] for s in 1:3 for t in 1:T)))
end


-------------------------------------------
number of total shifts for nurse_1 is: 6.0
number of total shifts for nurse_2 is: 5.0
number of total shifts for nurse_3 is: 4.0
number of total shifts for nurse_4 is: 6.0
number of total shifts for nurse_5 is: 4.0
number of total shifts for nurse_6 is: 6.0
number of total shifts for nurse_7 is: 7.0
number of total shifts for nurse_8 is: 7.0
number of total shifts for nurse_9 is: 6.0
number of total shifts for nurse_10 is: 7.0
number of total shifts for nurse_11 is: 7.0
number of total shifts for nurse_12 is: 7.0
number of total shifts for nurse_13 is: 6.0
number of total shifts for nurse_14 is: 7.0
number of total shifts for nurse_15 is: 7.0


In [44]:
#Is the number of N nurses  enough to satisfy the demand?
println("-------------------------------------------")
if value.(sum(alpha)) >= 0.000001
    println("NO! The number of nurses is not enough to satisfy the demand")
else
    println("YES! The number of nurses is enough to satisfy the demand")
end


-------------------------------------------
NO! The number of nurses is not enough to satisfy the demand


In [45]:
#How many hours does each nurse work?
println("-------------------------------------------")
for i in 1:N
    println("nurse_", i, " works for: ", value.(sum(x[i,s,t]*h[s] for s in 1:3 for t in 1:T)), " hours.")
end



-------------------------------------------
nurse_1 works for: 44.0 hours.
nurse_2 works for: 40.0 hours.
nurse_3 works for: 36.0 hours.
nurse_4 works for: 44.0 hours.
nurse_5 works for: 36.0 hours.
nurse_6 works for: 46.0 hours.
nurse_7 works for: 51.0 hours.
nurse_8 works for: 51.0 hours.
nurse_9 works for: 47.0 hours.
nurse_10 works for: 54.0 hours.
nurse_11 works for: 54.0 hours.
nurse_12 works for: 54.0 hours.
nurse_13 works for: 48.0 hours.
nurse_14 works for: 52.0 hours.
nurse_15 works for: 52.0 hours.


In [46]:
println("-------------------------------------------")
#Maximum number of total hours worked
println()
println("Maximum number of total hours worked: ", maximum(value.(sum((x[:,s,t])*h[s] for s in 1:3 for t in 1:T))))

-------------------------------------------

Maximum number of total hours worked: 54.0


In [47]:
########The End!